In [1]:
# Set the Seed Value for the Notebook so the Results are Reproducible
from numpy.random import seed
seed(1)

In [ ]:
# Import Modules & Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import tensorflow
import pandas as pd

In [ ]:
# Import Modules & Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Read in CSV File
df = pd.read_csv("Data/personality_traits_adjusted_scale_MOSHER.csv")

df.head()

In [ ]:
# Original 1-5 Likert Scale Features
# heights_features = df[["Funniness", "Giving", "Compassion_to_animals", "Loneliness", "God", "Dreams", "Children",
#                        "Getting_angry", "Energy_levels", "Getting_up", "Questionnaires_or_polls"]]

# snakes_features = df[["Reliability", "Funniness", "Criminal_damage", "Hypochondria", "Giving", "Compassion_to_animals",
#                       "Dreams", "Waiting", "Socializing", "Achievements", "Happiness_in_life", "Energy_levels",
#                       "Finding_lost_valuables"]]

# public_speaking_features = df[["Writing_notes", "Health", "God", "New_environment", "Life_struggles", "Finding_lost_valuables",
#                                "Parents_advice"]]

# ageing_features = df[["Writing_notes", "Criminal_damage", "Elections", "Charity", "Socializing", "Public_speaking",
#                       "Energy_levels"]]

# spiders_features = df[["Friends_versus_money", "Decision_making", "Empathy", "Eating_to_survive", "Giving",
#                        "Compassion_to_animals", "Loneliness", "Dreams", "New_environment", "Children",
#                        "Knowing_the_right_people", "Life_struggles", "Happiness_in_life", "Energy_levels", "Small_big_dogs",
#                        "Personality"]]

In [ ]:
# Adjusted 1-3 Likert Scale Features
adj_heights_features = df[["Funniness", "Loneliness", "Energy_levels", "Knowing_the_right_people", "Eating_to_survive",
                           "Waiting", "Life_struggles", "Appearence_and_gestures", "Self_criticism", "Health"]]

adj_snakes_features = df[["Health", "Public_speaking", "Thinking_ahead", "Getting_angry", "God", "Number_of_friends",
                          "Children"]]

adj_public_speaking_features = df[["Health", "God", "New_environment", "Parents_advice", "Appearence_and_gestures",
                                   "Assertiveness", "Socializing", "Hypochondria", "Changing_the_past", "Friends_versus_money"]]

adj_ageing_features = df[["Health", "Appearence_and_gestures", "God", "Compassion_to_animals", "Cheating_in_school",
                          "Life_struggles", "Knowing_the_right_people", "Giving", "Waiting", "Finding_lost_valuables",
                          "Empathy", "Judgment_calls"]]

adj_spiders_features = df[["Number_of_friends", "Children", "Life_struggles", "Happiness_in_life", "Public_speaking", 
                           "Questionnaires_or_polls", "Getting_angry", "Appearence_and_gestures", "Getting_up",
                           "Unpopularity"]]

In [ ]:
# Define the Features (X) and Target (y) to be Used in the Model
X = adj_snakes_features
y = df["Snakes"]
print(X.shape, y.shape)

In [ ]:
# Use Train_Test_Split to Create Training and Testing Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [ ]:
# Create a MinMaxScaler Model and Fit it to the Training Data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the Training and Testing Data Using the X_Scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Label-Encode Data
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Convert Encoded Labels to One-Hot-Encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
# Create a Normal Neural Network Model and Add Layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim= (X.shape[1])))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Compile and Fit the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(predictions)
pd.DataFrame({"Prediction": prediction_labels, "Actual": y_test}).head(50)

In [ ]:
# model.save("heights_model_trained.h5")
# model.save("snakes_model_trained.h5")
# model.save("public_speaking_model_trained.h5")
# model.save("ageing_model_trained.h5")
model.save("models/spiders_model_trained.h5")